# Planar Straight-Line Graph and Doubly-Connected Edge List

## Определения

**ППЛГ** (англ. _PSLG_) — Плоский (планарный) прямолинейный граф. Граф называется планарным, если его можно нарисовать на плоскости так, чтобы его ребра не пересекались. Планарный граф называется прямолинейным, если каждое его ребро — это прямая линия.


**Грань** (англ. _face_) — плоский граф разбивает плоскость на несколько областей, называемых гранями. Одна из граней не ограничена, ее называют внешней (англ. _external_) гранью, а остальные — внутренними (англ. _internal_) гранями.

**Разбиение плоскости** (англ. _subdivision_) — некоторе разделение плоскости на области. Заметим, что любой плоский граф разбивает плоскость на несколько областей (граней). Одна из граней не ограничена, ее называют внешней (англ. _external_) гранью, а остальные — внутренними (англ. _unbounded_) гранями.


**РСДС** (англ. _DCEL_) — реберный список с двойными связями. Реберный список с двойными связями предназначен для представления ППЛГ.

<center><img src="pictures/planar_graph.png" width="300"></center>
_Пример планарного графа. Синим контуром обозначены грани, за исключением внешней грани (всего 5 граней). Обратите внимание, что внутри грани могут содержаться другие ребра и вершины._

## Описание РСДС

### HalfEdge

In [8]:
class HalfEdge:
    def __init__(self, origin, twin, nxt, prv, incident_face):
        self.origin = origin #Вершина из которой выходит полуребро
        self.twin = twin #Полуребро-близнец, направленное в противоположную сторону
        self.nxt = nxt #Первое ребро, идущее вслед за данным против часовой стрелки
        self.prv = prv #Ребро, для которого nxt - данное ребро
        self.incident_face = incident_face #Инцидентная данному полуребру грань
        #Пара полуребер this и twin представляет изначальное ребро графа.

### Vertex

In [9]:
class Vertex:
    def __init__(self, geometry, incident_edge):
        self.geometry = geometry #Описание вершины в пространстве
        self.incident_edge = incident_edge #Инцидентное полуребро

### Face

In [10]:
class Face:
    def __init__(self, internal, external):
        self.external = external #Полуребро на границе грани
        self.internal = internal #Список полуребер внутренних граней (по одному на каждую грань)

_Пример РСДС:_
<center><img src="pictures/dcel.png" width="520"></center>

## Применение 

### Пример: получить смежные грани для ребра

### Пример: перебрать все ребра инцидентные данной вершине по часовой стрелке

# Построение РСДС множества прямых

Пусть  $L$ $-$ множество прямых на плоскости. 

Множество $L$ индуцирует разбиение плоскости, состоящее из вершин, ребер и граней (некоторые из которых могут быть неограниченны). Такое разбиение обозначается $A(L)$ и называется _конфигурацией (arrangement_).

Назовем конфигурацию _простой (simple)_ если никакие три прямые не пересекаются в одной точке и никакие две прямые не являются параллельными.

_Сложностью (complexity)_ конфигурации назовем общее число вершин, ребер и граней в ней.

>Утверждение:
Пусть $L$ $-$ множество прямых, $L = \{l_1, ..., l_n\}$. $A(L)$ $-$ кофигурация $L$. Тогда 
1. Число вершин в $A(L)$ не превышает $\frac{n(n - 1)}{2}$
2. Число ребер $A(L)$ не превышает $n^2$
3. Число граней $A(L)$ не превышает $\frac{n^2}{2} + \frac{n}{2} + 1$
<div>
_Во всех трех утверждениях равенство достигается тогда и только тогда, когда конфигурация $A(L)$ простая._

$\triangleright$

1) Так как вершинами $A(L)$ являются точки пересечения пар прямых из $L$, их количество не может быть более чем $\frac{n(n - 1)}{2}$. 

2) Число ребер на одной прямой на единицу больше, чем число вершин на ней, а число вершин на прямой не превышает $n - 1$. Следовательно, число ребер на одной прямой ограничено сверху величиной $n$. Поэтому число ребер всегда не более $n^2$.

3) Чтобы оценить число граней в $A(L)$ будем добавлять прямые по одной и следить за увеличением количества граней на каждом шаге. $\forall i: 1 \leq i \leq n$ определим $L_i := \{l_1, ..., l_i\}$. Каждое ребро на $l_i$ разбивает какую-то грань $A(L_{i-1})$ на две, поэтому число граней увеличивается на число ребер $A(L_{i-1})$ на прямой $l_i$. Но это число не превышает $i$, поэтому число граней не может быть больше 
<center>$$1 + \sum_{i=1}^n i = \frac{n^2}{2} + \frac{n}{2} + 1$$</center>

$\triangleleft$

## Алгоримт построения

##### Заметающая прямая

Поскольку количество точек пересечения растет квадратично, алгоритм будет работать за $O(n^2log(n))$

##### Инкрементальный алгоритм

Инкрементальный алгоритм добавляет прямые $l_1, l_2, ..., l_n$ по одной и на каждой итерации обновляет РСДС.

Обозначим за $A_i$ разбиение плоскости, индуцированное $A(\{l_1, ..., l_i\})$. 

Чтобы добавить $l_i$ необходимо разбить грани конфигурации $A_{i-1}$, которые пересекает $l_i$. Чтобы найти эти грани, пройдем по уже существующим полуребрам вдоль $l_i$ слева направо, попутно разбивая пересекаемые грани. 

Разбиение грани _f_: создаем две новые записи о гранях: по одной для частей _f_ выше и ниже $l_i$. Проходим вдоль границы _f_, следуя по указателям _next_, пока не достигнем полуребра соответствующего ребра _e'_, где $l_i$ покидает _f_. 

Разбиваем ребро _e'_, через которое $l_i$ покидает _f_ и создаем новую вершина для $l_i \cap e'$ и добавим записи о полуребрах $l_i \cap f$.  

Общее время разбиения линейно зависит от сложности _f_.

_Разбиение грани:_
<center><img src="pictures/splitting.png" width="525"></center>

Заметим, что при проходе вдоль $l_i$ мы можем покинуть очередную грань только двумя способами

1) Пусть мы покидаем _f_ по некоторому ребру _e'_. Тогда воспользуемся указателем _twin_ этого полуребра, чтобы добраться до другого полуребра _e'_ в РСДС. Таким образом, мы найдем следующую грань за время, пропорциональное сложности _f_. 

2) Пусть мы покидаем _f_ через некоторую вершину _v_. Тогда совершим обход вокруг _v_, посещая все инцидентные ей ребра, пока не найдем следующую грань, которую пересекает $l_i$. РСДС позволяет сделать это за время, пропорциональное степени вершины _v_.

Чтобы найти самое левое ребро, пересекающее $l_i$, с каждой добавленной прямой будем хранить множество полуребер, на которые разбита данная прямая, упорядоченных по _origin.geometry_. При добавлении прямой $l_i$ пересечем ее со всеми ранее добавленными прямыми и найдем самую левую точку пересечения. Пусть эта точка лежит на прямой $l_j$. Теперь осталось найти полуребро _e_ такое, что $e.origin.geometry \le p.geometry$ и $e.twin.origin.geometry \gt p.geometry$, где _p_ $-$ найденная точка пересечения прямых $l_i$ и $l_j$.

_Обход конфигурации:_
<center><img src="pictures/traversing-arrangement.png" width="400"></center>

## Оценка времени работы

Для оценки времени разбиения граней введем понятие _зоны_. 

_Зоной_ прямой $l$ в конфигарации $A(L)$ называется множество граней $A(L)$, замыкания которых пересекают $l$. 

_Сложность_ зоны определяется как суммарная сложность всех составляющих ее граней (то есть, общее количество ребер и вершин в грани). Время, необходимое для добавления прямой $l_i$ линейно зависит от сложности зоны $l_i$ в конфигурации $A(\{l_1, ..., l_i\})$.

_Зона прямой в конфигурации:_
<center><img src="pictures/zone.png" width="500"></center>

>Утверждение:
Сложность зоны прямой $l$ в конфигурации $m$ прямых на плоскости равна $O(m)$.

$\triangleright$

Пусть $L = \{l_1, ..., l_m\}$ $-$ множество $m$ прямых на плоскости, $l$ $-$ еще одна прямая. Без потери общности, можем считать, что $l$ совпадает с осью $x$ (в противном случае можем заменить систему координат).

Каждое ребро в $A(L)$ инцидентно двум граням. Назовем ребро _левым граничным_ для грани, примыкающей к нему справа и _правым граничным_ для грани, примыкающей слева. Для доказательства утверждения достаточно показать, что количество левых граничных ребер (и, соответственно, правых) в зоне $l$ не превышает $5m$.

Доказательство проведем индукцией по $m$.

В базовом случае, $m=1$, утверждение очевидно (одна прямая индуцирует зону, содержащую две неограниченные грани).

Рассмотрим случай $m \gt 1$. Из прямых, входящих в $L$, обозначим $l_1$ ту, точка пересечения которой с $l$ лежит правее всех остальных (если таких прямых несколько, то возьмем любую из них). По предположению индукции, зона $l$ в $A(L \setminus \{l_1\})$ содержит не более $5(m-1)$ левых граничных ребер. В результате добавления прямой $l_1$ количество левых граничных ребер может увеличиться двумя способами: 

1) Возникают новые левые граничные ребра на $l_1$, а старые левые граничные ребра разбиваются $l_1$ на части. Рассмотрим точки $v$ $-$ первая точка пересечения $l_1$ с другой прямой из $L$, расположенная выше $l$, $w$ $-$ первая точка пересечения, расположенная ниже $l$. Ребро, соединяющее $v$ и $w$, является новым левым граничным ребром на $l$.

2) Кроме того, $l_1$ разбивает существующее левое граничное ребро в точках $v$ и $w$. В результате количество левых граничных ребер увеличивается на $3$ Если не существует таких $v$ и $w$, то увеличение будет еще меньше.

Таким образом, количество левых граничных ребер может увеличиться не более чем на $3$. В этом случае общее число левых граничных ребер не превышает $5(m-1)+3 \lt 5m$.

$\triangleleft$

_Пример нового левого граничного ребра:_
<center><img src="pictures/bounding-edges.png" width="400"></center>

>Утверждение:
Сложность инкрементального алгоритма $O(n^2)$.

$\triangleright$

Ограничим сверху время работы инкрементального алгоритма построения конфигурации. Так как время, необходимое для вставки $l_i$ линейно зависит от сложности зоны $l_i$ в $A(\{l_1, ..., l_{i-1}\})$, то по доказанному утверждению время вставки всех прямых равно
<center><div>
$$\sum_{i=1}^n O(i) = O(n^2)$$
</div></center>

Таким образом, сложность каждого шага алгоритма не более чем квадратична, следовательно сложность всего алгоритма построения $O(n^2)$.

$\triangleleft$